## What are these CSDS files?

In this notebook, you will download the sample CSDS data for 4 matches. CSDS stands for Counter Strike Data Science. We extract data from each demo file into outputs called channels. Each channel is one of four types.

- Telemetry: Information as a function of time
- Single event: Information for a single event type
- Multi event: Information for several events of similar type grouped together (For example, bomb events)
- Header: Metadata about the map such as map_name or tick_rate

There are 33 channels, and you can see more information about each in the manifest file. They are all compressed parquet files except for the manifest, which is gzipped json. When we read in a CSDS file, we generally don't need all channels, so our reader will accept "instructions" that describe which channels and columns to read. An example of some instructions are in step 5.

Each demo goes through our parser then through a post-parser processor (PPP) that fixes certain things and extends some channels with info from other channels. For instance, most events have a `player_id` attached to them, but this ID can be two different numbers for the same person if they disconnect and reconnect to the server. Therefore we added `player_id_fixed` which is tied to steam id and does not change even if a player disconnects or reconnects.

## Downloading the data

We built in a simple downloader for the sample data. The properties of the sample data are:

- 4 matches
- ~120 files 
- ~100mb
- ~10 seconds to download

For the main dataset of 60k+ matches, you need to subscribe to the [product on the AWS Data Exchange](https://aws.amazon.com/marketplace/pp/prodview-v3o7zrt6okwmo).

The use license will be in your `PURESKILLGG_TOME_DS_COLLECTION_PATH` named `license.pdf`. You must agree to these terms to use the data.

Note: you can safely stop and restart this downloader.

In [ ]:
from pureskillgg_makenew_pyskill.notebook import setup_notebook

In [ ]:
# Set environment variables
setup_notebook()

In [ ]:
import os
import requests
import zipfile

In [ ]:
# # Choose our Data Science collection path as the location to save
dataset_sample_urls = [
    "https://d1ewbp317vsrbd.cloudfront.net/91b4d9a6-26a7-4c2e-838f-ba8d86c2b8b2.zip",
    "https://d1ewbp317vsrbd.cloudfront.net/4f6a3d09-99e8-476a-ba30-c7ee1992a276.zip"
]

ds_collection_path = os.environ.get('PURESKILLGG_TOME_DS_COLLECTION_PATH')

for index, url in enumerate(dataset_sample_urls):
    output_filename = os.path.join(ds_collection_path,f"sample_{index}.zip")
    if os.path.exists(output_filename):
        continue

    response = requests.get(url)
    with open(output_filename, "wb") as f:
        f.write(response.content)
        
    with zipfile.ZipFile(output_filename, 'r') as zip_ref:
        zip_ref.extractall(ds_collection_path)